# Il più semplice esempio di interazione con modelli locali via l'API di OpenAI

Luca Mari, gennaio 2025  

Quest'opera è distribuita con <a href="http://creativecommons.org/licenses/by-nc-sa/4.0" target="_blank">Licenza Creative Commons Attribuzione - Non commerciale - Condividi allo stesso modo 4.0 Internazionale</a>.  
<img src="https://creativecommons.it/chapterIT/wp-content/uploads/2021/01/by-nc-sa.eu_.png" width="100">

**Obiettivo**: comprendere le più semplice modalità di interazione con modelli locali via l'API di OpenAI.  
**Precompetenze**: basi di Python.

> Per eseguire questo notebook con VSCode sul proprio calcolatore, occorre:
> * installare un interprete Python
> * attivare un server locale che renda possibile l'interazione con un modello via l'API di OpenAI (per semplicità, supporremo che sia LM Studio, scaricabile da https://lmstudio.ai)
> * scaricare da https://code.visualstudio.com/download e installare VSCode
> * eseguire VSCode e attivare le estensioni per Python e Jupyter
> * ancora in VSCode:
>     * creare una cartella di lavoro e renderla la cartella corrente
>     * copiare nella cartella il file di questa attività: [oaibase.ipynb](oaibase.ipynb)
>     * aprire il notebook `oaibase.ipynb`
>     * creare un ambiente virtuale locale Python (Select Kernel | Python Environments | Create Python Environment | Venv, e scegliere un interprete Python):
>     * installare i moduli Python richiesti, eseguendo dal terminale:  
>         `pip install openai`

Importiamo i moduli Python necessari e specifichiamo l'_end point_ per l'accesso al server locale.

In [1]:
from openai import OpenAI
from pprint import pprint
from IPython.display import Markdown, display
import json

client = OpenAI(base_url="http://localhost:1234/v1") # usa un server locale, per esempio con LM Studio  

### Elenco dei modelli disponibili
Assumendo che il server sia attivo, questo è il più semplice esempio di una richiesta al server via l'API di OpenAI, per ottenere la lista dei modelli accessibili attraverso il server stesso.

In [2]:
models = client.models.list()
pprint(models.data)

[Model(id='qwen2.5-14b-instruct', created=None, object='model', owned_by='organization_owner'),
 Model(id='deepseek-r1-distill-llama-8b', created=None, object='model', owned_by='organization_owner'),
 Model(id='deepseek-r1-distill-qwen-7b', created=None, object='model', owned_by='organization_owner'),
 Model(id='text-embedding-nomic-embed-text-v1.5@q4_k_m', created=None, object='model', owned_by='organization_owner'),
 Model(id='phi-4', created=None, object='model', owned_by='organization_owner'),
 Model(id='text-embedding-granite-embedding-278m-multilingual', created=None, object='model', owned_by='organization_owner'),
 Model(id='llama-3.3-70b-instruct', created=None, object='model', owned_by='organization_owner'),
 Model(id='llama-3.2-1b-instruct', created=None, object='model', owned_by='organization_owner'),
 Model(id='llama-3.2-3b-instruct', created=None, object='model', owned_by='organization_owner'),
 Model(id='mathstral-7b-v0.1', created=None, object='model', owned_by='organiza

### Generazione di sentence embedding
Assumendo che sia stato caricato un modello di _embedding_ sul server, possiamo usarlo per fare _sentence embedding_. 

In [ ]:
text = "La bellezza salverà il mondo."

response = client.embeddings.create(
    model="...",
    input=[text]
).data[0].embedding

print(f'''La frase è stata codificata in un vettore di {len(response)} numeri;
i primi 5 sono: {response[:5]}''')

La frase è stata codificata in un vettore di 768 numeri;
i primi 5 sono: [-0.07184145599603653, -0.011897610500454903, -0.023683611303567886, -0.010468722321093082, 0.011160383932292461]


### Completion
Questo è il più semplice esempio di una richiesta al server via l'API di OpenAI (stiamo supponendo che il server sia attivo e sia stato caricato un modello).

In [3]:
prompt = "Presentati: chi sei?"

response = client.chat.completions.create(
    model="...",
    messages=[
        {"role": "system", "content": "Rispondi sempre in italiano."},
        {"role": "user", "content": prompt}
    ],
    max_tokens=-1,
    temperature=0.7
)

print("L'intero oggetto JSON di risposta:")
pprint(dict(response))

print("\nIl messaggio generato:")
display(Markdown(response.choices[0].message.content))

L'intero oggetto JSON di risposta:
{'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sono un'intelligenza artificiale creata da Alibaba Cloud per assistere, informare e conversare con gli utenti come te su una vasta gamma di argomenti. Mi chiamano Qwen, sono felice di incontrarti! Come posso aiutarti oggi?", refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))],
 'created': 1737802311,
 'id': 'chatcmpl-bu85rmnqouedq3ww4j7nlg',
 'model': 'qwen2.5-14b-instruct',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'qwen2.5-14b-instruct',
 'usage': CompletionUsage(completion_tokens=62, prompt_tokens=27, total_tokens=89, completion_tokens_details=None, prompt_tokens_details=None)}

Il messaggio generato:


Sono un'intelligenza artificiale creata da Alibaba Cloud per assistere, informare e conversare con gli utenti come te su una vasta gamma di argomenti. Mi chiamano Qwen, sono felice di incontrarti! Come posso aiutarti oggi?

### Completion con structured output
Solo un poco più complessa è una richiesta in cui si specifica lo schema JSON che vogliamo sia utilizzato nella risposta, nuovamente in accordo all'API di OpenAI, nella specifica _structured output_ (https://platform.openai.com/docs/guides/structured-outputs).

In [10]:
response_format = {
    "type": "json_schema",
    "json_schema": {
        "name": "presentazione",
        "strict": "true",
        "schema": {
            "type": "object",
            "properties": {
                "nome": {"type": "string"},
                "caratteristiche principali": {"type": "string"},
                "caratteristiche secondarie": {"type": "string"}
            },
        "required": ["nome", "caratteristiche principali"]
        }
    }
}

response = client.chat.completions.create(
    model="...",
    messages=[
        {"role": "system", "content": "Rispondi sempre in italiano." },
        {"role": "user", "content": "Presentati: chi sei?" }
    ],
    max_tokens=-1,
    temperature=0.7,
    response_format=response_format # type: ignore
)

print("L'intero oggetto JSON di risposta:")
pprint(dict(response))

print("\nIl messaggio generato:")
pprint(json.loads(response.choices[0].message.content)) # type: ignore

L'intero oggetto JSON di risposta:
{'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{ "nome": "Phi", "caratteristiche principali": "Sono un modello linguistico di intelligenza artificiale sviluppato da Microsoft, progettato per comprendere e generare testo in modo coerente ed efficace. Le mie capacità includono la risposta a domande, la fornitura di spiegazioni su una varietà di argomenti, l\'assistenza nella comprensione di concetti complessi e il supporto nell\'apprendimento e nella risoluzione dei problemi. Sono in grado di offrire informazioni accurate fino alla mia ultima aggiornamento nel 2023. Tuttavia, è importante notare che non ho accesso a dati o eventi successivi a tale data." }', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))],
 'created': 1737731736,
 'id': 'chatcmpl-bpppqh3463njxp5njduom8',
 'model': 'phi-4',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint

### Function calling

https://platform.openai.com/docs/guides/function-calling 

In [62]:
def compute_sqrt(number: float) -> float:   # la funzione da chiamare
    return number ** 0.5

tools = [                                   # la definizione della funzione
    {
        "type": "function",
        "function": {
            "name": "compute_sqrt",
            "description": "Calcola la radice quadrata di un numero",
            "parameters": {
                "type": "object",
                "properties": {
                    "number": {
                        "type": "float",
                        "description": "Il numero di cui calcolare la radice quadrata"
                    }
                },
                "required": ["number"]
            }
        }
    }
]

prompt = "Calcola la radice quadrata di 64."
messages=[
    {"role": "system", "content": "Rispondi sempre in italiano." },
    {"role": "user", "content": prompt }
]

response = client.chat.completions.create(
    model="...",
    messages=messages, # type: ignore
    max_tokens=-1,
    temperature=0.7,
    tools=tools # type: ignore
)

In [63]:
print("L'intero oggetto JSON di risposta:")
pprint(dict(response))

print("\nIl messaggio generato:")
pprint(response.choices[0].message)

if response.choices[0].message.tool_calls:
    tool_call = response.choices[0].message.tool_calls[0]

    print("\nLa parte del messaggio riferita alla funzione da chiamare:")
    print(tool_call.function)

    function_name = tool_call.function.name
    function_arguments = json.loads(tool_call.function.arguments)
    print(f"\nLa funzione da chiamare è: {function_name}; i suoi argomenti sono: {function_arguments}")

L'intero oggetto JSON di risposta:
{'choices': [Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='534472074', function=Function(arguments='{"number":64}', name='compute_sqrt'), type='function')]))],
 'created': 1736889005,
 'id': 'chatcmpl-eaxj11o7e2mhrcb5qgiom',
 'model': 'qwen2.5-14b-instruct',
 'object': 'chat.completion',
 'service_tier': None,
 'system_fingerprint': 'qwen2.5-14b-instruct',
 'usage': CompletionUsage(completion_tokens=22, prompt_tokens=188, total_tokens=210, completion_tokens_details=None, prompt_tokens_details=None)}

Il messaggio generato:
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='534472074', function=Function(arguments='{"number":64}', name='compute_sqrt'), type='function')])

La parte del messaggio r

In [64]:
to_evaluate = function_name + "(" + str(function_arguments['number']) + ")"
result = eval(to_evaluate)

print(f"\nIl risultato della funzione è: {result}")


Il risultato della funzione è: 8.0


In [65]:
prompt_2 = "Quanto vale la radice quadrata di 64?"
messages.append({"role": "user", "content": prompt_2})
messages.append({"role": "assistant", "function_call": {"name": function_name, "arguments": function_arguments}}) # type: ignore
messages.append({"role": "tool", "content": str(result)})

response_2 = client.chat.completions.create(
    model="...",
    messages=messages, # type: ignore
    tools=tools # type: ignore
)

In [66]:
print(response_2.choices[0].message.content)

La radice quadrata di 64 è 8.0.
